### Importing necessary libraries

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
import pickle
import numpy as np
from keras.models import load_model
import json
import random
import tkinter
from tkinter import *

### Loading the pretrained model,saved pickle files

In [2]:
## Initializing lemmatizer and loading the pre-trained model, intents and words..
lemmatizer = WordNetLemmatizer()
model = load_model('chatbot_model.h5')
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

### preprocessing the user data

In [3]:
def clean_up_sentence(sentence):
    # tokenizing the sentences(spliiting the content into individuals)
    sentence_words = nltk.word_tokenize(sentence)
    
    # stemming each word (shrinking the words to base form)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [4]:
# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words):
    
    # calling the cleanup function to tokenize and lemmatize the pattern
    sentence_words = clean_up_sentence(sentence)
    
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                
                # will assign 1 if current word is in the vocabulary position
                bag[i] = 1
    return(np.array(bag))

### predicting the result tag with pre trained model

In [5]:
#Returns a list of intents (classes) with associated probabilities that the sentence belongs to..

def predict_class(sentence, model):
    
    # filtering out the predictions below a threshold
    p = bow(sentence, words)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

### returning the response according to the tag

In [6]:
#Returns a random response corresponding to the highest-probability intent..
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result       

In [7]:
#returns the result
def chatbot_response(text):
    try:
     ints = predict_class(text, model)
     res = getResponse(ints, intents)
     return res

    except Exception as e:
     return "can't understand check the input and try again"

### Creating a GUI with tkinter

In [19]:
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    
    
    if msg not in['',"asde","hg","qw","z3e",'asd','gyt']:
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#17202A", font=("raleway", 11 ))
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
    else:
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#17202A", font=("raleway", 11 ))
        ChatLog.insert(END, "Bot: " + "invalid input check your input and try again" + '\n\n')
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
        
        
base = Tk()
base.title("BOT")
base.geometry("500x500")
base.resizable(width=FALSE, height=FALSE)

#Creating Chat window
ChatLog = Text(base, bd=0, bg="#EBF5FB", height="6", width="800", font="raleway",)
ChatLog.config(state=DISABLED)

#Binding scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Creating Button to send message
SendButton = Button(base, font=("raleway",12,'bold'), text="Send", width="12", height="5", bd=0, bg="#2E86C1", activebackground="#F4D03F",fg='#ffffff', command= send )

#Creating the box to enter message
EntryBox = Text(base, bd=0, bg="#5D6D7E",width="29", height="5",fg='#ffffff', font="raleway",insertbackground='white',insertwidth=5

#Placing all components on the screen
scrollbar.place(x=471,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=470)
EntryBox.place(x=135, y=401, height=90, width=338)
SendButton.place(x=6, y=401, height=90,width=120)
base.mainloop()